In [1]:
import csv
import pymongo
from tkinter import *
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
from pymongo.errors import BulkWriteError
import pandas as pd
import numpy as np
import json
import datetime
import os
from bson.json_util import loads,dumps
import bson.json_util as js_utils
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

#### classes for parsing and mongodb CRUD operations

In [2]:
#the client wants UK airportdata only so non-UK airports is excluded    

class Validation(object):
    def __init__(self,csvFile):
        self.csvFile=csvFile
        
    ###validation check to ensure the csv data is coming from 1 of 3: airport-frequencies,airports or runways
    
    def validation_check(self,criteria):
       
        checkList1=[]
        control=0
        for line in self.csvFile:
            while control < 1:
                checkList1.append(line.keys())
                control+=1
        
        checkList2=[]
        for item in checkList1[0]:
            checkList2.append(item)

        valnum=0
        for label in criteria:
            if label in checkList2:
                valnum+=1
            else:
                pass

        if valnum == 18 or valnum ==20 or valnum == 6:
            return True
        else:
            return False
        

    
     
    ###where the ids in the id's field in the csv is used as the id in mongo, so that mongo doesn't autogen
    ###ids. This prevents duplication of documents
class Mongo_prep(object):
    def __init__(self,csvFile=None):
        self.csvFile=csvFile
       
        
    def iD(self):
        listcon=[]
        for line in self.csvFile:
            line['_id']=line.pop('id')
            listcon.append(line)
        return listcon
    
    def surrogateKey(self,df):
        df.rename(columns = {'_id':'id'}, inplace = True)
        df['PlaceHolderID'] =df['frequency_mhz']
        df['PlaceHolderID'] = df['PlaceHolderID'].replace(np.NaN,"0")
        df['_id']=("ICAO#"+df["ident"])+("Freq"+df["PlaceHolderID"])
        df.drop(columns='PlaceHolderID',inplace=True)
    
        return df

               
 #adding columns needed, and included small_airport,medium_airport,large_airport
   
class MongoDB_controls(object):
    def __init__(self,client):
        self.client = pymongo.MongoClient(client)
            
#     def mongo_start(self):
#         client = pymongo.MongoClient("mongodb://localhost:27017/")
#         return client
        
    def mongo_db(self,db):
        db=self.client[db]
        return db
        
    def mongo_collection(self,db,collection):
        collection=db[collection]
        return  collection
 
 #inserting functions          
    def mongo_insertMany(self,collection,file):
        collection.insert_many(file)
      
  #Needs testing    
    def mongo_insertOne(self,collection,file):
        collection.insert_one(file)
        
    def mongo_call(self):
        return self.client.list_database_names()
        
    def mongo_find(self,collection,rows=" "):
        call=[]
        if rows==" ":
            result = collection.find({})
            for r in result:
                call.append(r)
            return call
        else:
            result = collection.find(rows)
            for r in result:
                call.append(r)
            return call
        
   #delete block needs testing
    def mongo_deleteOne(self,collection,del_record):
        collection.delete_one(del_record)
            
    def mongo_deleteMany(self,collection,del_records):
        collection.delete_many(del_records)
         
    def mongo_DeleteAll(self,collection):
            collection.delete_many({})
        
    #update block
    def mongo_updateOne(self,collection,iD,data):
        collection.update_one({'id':iD},{"$set":data})
        
        

#### Cleaning dataset using pandas

In [3]:
##this class is there to prep the airport data
class Pandas_airport(object):
    def __init__(self,df):
        self.df=df
        
    def dropTypeClose(self):
        df=self.df
        df.drop(df.index[df['type']=='closed'],inplace=True)
        df.reset_index(inplace=True)
        
     #creating blank columns equal to the number of unique values found under type   
    def airportTypeColumns(self):
        df=self.df
        df[df.type.unique()]=""
        for uniquedf in df.type.unique():
            df[uniquedf]=df['type']


    
    def marker(self,aiportType):
        df = self.df
        removeVal=[]
        keepVal=[]
        for unique in df.type.unique():
            if unique == aiportType:
                keepVal.append(unique)
            else:
                removeVal.append(unique)
        
        ### the following code is used to place a place holder "np.NaN" needed to copy across freq data
        
        df[aiportType] = df[aiportType].replace(removeVal,'')
        df[aiportType] = df[aiportType].replace(keepVal,np.NaN)
        
    ## this block keeps GB airports columns plus small_airport, medium_airport and large_airport types & columns and drops the 
    ## rest. Filtered on iso_country as these codes are unique to each country
    
    def sizeNGB_sec(self):
        df = self.df
        df.drop(df.index[df['iso_country']!='GB'],inplace=True)
        df=df.filter(items=["_id","ident","type","name","latitude_deg","longitude_deg",
                         "elevation_ft","continent","iso_country" ,"iso_region",
                         "municipality","scheduled_service","iata_code","local_code",
                          "home_link","wikipedia_link","keywords",
                         "small_airport","medium_airport","large_airport"])
        
        df.reset_index(inplace=True)
    
        return df

## this class is there to prep the freq data

class Pandas_freq(object):
    def __init__(self,df):
        self.df=df
        
    def indexSet(self,column='airport_ident'):
        df = self.df
        df.set_index(column,inplace=True)
        return df
    
    
    def freqselect(self):
        df = self.df
        df=df['frequency_mhz']
        return df
        

        

class MergingAnF(object):
    def __init__(self,airport,data):
        self.airport=airport
        self.data=data
    
    def mergeAnF(self):
        mergedAnF=pd.merge(self.airport,self.data,left_on='ident',right_on='airport_ident',how='left')
        return mergedAnF

class Copying_freq(object):
    def __init__(self,combined_data):
        self.combined_data=combined_data
       
    def copyfreq(self,airportsize,freqmhz='frequency_mhz'):
        self.combined_data[airportsize] = self.combined_data[airportsize].fillna(self.combined_data[freqmhz])
    
    def returnDataFrame(self):
        return self.combined_data

#### Json backup and Restore

In [4]:
##This block is used to backup the prepped data in .json format
class Js_func(object):
    def __init__(self,file=None,filename=None):
        self.file=file
        self.filename=filename
    
    def js_backup(self):
        name=self.filename
        dateTimeObj=datetime.datetime.now()
        date = dateTimeObj.strftime("%d-%b-%Y (%H%M%S)")
        outFile=open(name+" "+date+".json","w")
        json.dump(self.file,outFile)
        outFile.close()
        showinfo("Saved directory",os.getcwd())
    
    def js_restore(self):
        preppedColl.delete_many({})
        inFile=open(self.file,'r')
        dataFile=json.loads(inFile.read())
        dataFile_loaded=loads(dataFile)
        inFile.close()
        return dataFile_loaded

#### MongoDB
this pilot version of the software will require the user to start MongoDB's shell before running the program and connect to a local host -  mongodb://localhost:27017/

In [5]:
# these are default setting for the pilot. Database: aviation, Collection: airports,runway, freq 
mongo=MongoDB_controls("mongodb://localhost:27017/")

# naming the database. Name given aviation
aviationDB=mongo.mongo_db("aviation")

#creating collections. At the moment this is hardcoded but can be changed for the production version 
airportColl=mongo.mongo_collection(aviationDB,"airportColl")
runwayColl=mongo.mongo_collection(aviationDB,"runwaysColl")
freqColl=mongo.mongo_collection(aviationDB,"freqColl")
preppedColl=mongo.mongo_collection(aviationDB,"airFreqColl")
preppedCollincRunway=mongo.mongo_collection(aviationDB,"airFreqRunColl")

#### Adding collections to MongoDB

In [6]:
#inserting data into collections
def toDB(coll,data):
    mongo.mongo_insertMany(coll,data)

#### Core thread where the program comes together

In [7]:
def airportData(path="None",db=aviationDB,coll=airportColl):
    if path != None:
            ## the valgate passes a csv to the Validation class to check if the labels found in the csv match the target csv  
            ## if there is a match the data is is allowed in 
        valgate=""
        try:
            with open(path,'r',encoding='utf-8') as csv_file:
                airport = csv.DictReader(csv_file, delimiter=',')
                #the following code checks that the incoming csv contains the relevant labels
                airportlabels=['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg', 'elevation_ft', 
                                'continent', 'iso_country', 'iso_region', 'municipality', 'scheduled_service', 'gps_code', 
                                'iata_code', 'local_code', 'home_link', 'wikipedia_link', 'keywords']
                
                #this block is making a call to the validation class            
                airportval=Validation(airport)
                if airportval.validation_check(airportlabels)==True:
                    valgate=True
                else:
                    valgate=False
        
                if valgate==True:
                    try:
                        with open(path,'r',encoding='utf-8') as csv_file:
                            airport = csv.DictReader(csv_file, delimiter=',')
                            airportID=Mongo_prep(airport)
                            #### Needs checking but this is the entity relationship structure
                            air_ToDB_Prepped=airportID.iD()
                            toDB(coll,air_ToDB_Prepped)
                            showinfo("Imported","Airport data imported")
                        
                    #Main purpose is to inform the user that the .csv being imported is already in the database or 
                    #the .cvs contains id repeats that in the database
                    except BulkWriteError as bwe:
                        showinfo("Error",".csv already important/.csv has repeating IDs")
                        return(bwe.details)
                    
                    ###this error handling block is to caputre all other errors
                    except:
                        showinfo("Error","***Please contact administartor")
                else:
                    showinfo("Error","Validation checkes failed please check .csv labels")
        except FileNotFoundError:
            showinfo("Error","No file selected or file not found ")
            
        ###this error handling block is to caputre all other errors
        except:
            showinfo("Error", "***Please contact administartor")
     

       
                


def runwayData(path="None",db=aviationDB,coll=runwayColl):
    if path != None:
        
        valgate=""
        try:
            with open(path,'r',encoding='utf-8') as csv_file:
                runway = csv.DictReader(csv_file, delimiter=',')
                #the following code preps airport.csv for MongoDB
                
                runwayLabels=["id","airport_ref","airport_ident","length_ft","width_ft","surface",
                              "lighted","closed","le_ident","le_latitude_deg","le_longitude_deg","le_elevation_ft",
                              "le_heading_degT","le_displaced_threshold_ft","he_ident","he_latitude_deg","he_longitude_deg",
                              "he_elevation_ft","he_heading_degT","he_displaced_threshold_ft"]
                
                #this block is making a call to the validation class    
                runval=Validation(runway)
                if runval.validation_check(runwayLabels)==True:
                    valgate=True
                else:
                    valgate=False
                    print("Fail")
                
                
                #if the validation check passes the following code is run
                if valgate==True:
                    try:
                        with open(path,'r',encoding='utf-8') as csv_file:
                            runway = csv.DictReader(csv_file, delimiter=',')
                            runID=Mongo_prep(runway)
                            runway_ToDB_Prepped=runID.iD()
                            toDB(coll,runway_ToDB_Prepped)
                            showinfo("Imported","Runway data imported")
                        
                    #Main purpose is to inform the user that the .csv being imported is already in the database or 
                    #the .cvs contains id repeats that in the database
                    except BulkWriteError as bwe:
                        showinfo("Error",".csv already important/.csv has repeating IDs")
                        return(bwe.details)
                    
                else:
                    showinfo("Error","Validation checkes failed please check .csv labels")
                    
        except FileNotFoundError:
            showinfo("Error","No file selected or file not found ")
            
        ###this error handling block is to caputre all other errors
        except:
            showinfo("Error","Please contact administartor")
          
            

def freqData(path="None",db=aviationDB,coll=freqColl):
    if path != None:
        
        valgate=""
        try:
            with open(path,'r',encoding='utf-8') as csv_file:
                freq = csv.DictReader(csv_file, delimiter=',')
            
                #the following code checks the incoming csv contains the relevant labels
                airfreqlabels=['id','airport_ref','airport_ident','type','description','frequency_mhz']
                
                
                #this block is making a call to the validation class    
                freqval=Validation(freq)
                if freqval.validation_check(airfreqlabels)==True:
                    valgate=True
                else:
                    valgate=False
                    
                    
                #if the validation check passes the following code is run
                if valgate ==True:
                    try:
                        with open(path,'r',encoding='utf-8') as csv_file:
                            freq = csv.DictReader(csv_file, delimiter=',')
                
                            #the following code preps airport-frequencies.csv for MongoDB
                            airport_freq=Mongo_prep(freq)
                            freq_ToDB_Prepped=airport_freq.iD()
                            toDB(coll,freq_ToDB_Prepped)
                            showinfo("Imported","Frequency data imported")
                            
                    except BulkWriteError as bwe:
                        showinfo("Error",".csv already important/.csv has repeating IDs")
                        return(bwe.details)
                else:
                    showinfo("Error","Validation checkes failed please check .csv labels")
                    
        except FileNotFoundError:
            showinfo("Error","No file selected or file not found ")
            
        ###this error handling block is to caputre all other errors
        except:
            showinfo("Error","Please contact administartor")
            
def cleaningNmerging(airportcoll=airportColl,freqcoll=freqColl,db=aviationDB,coll=preppedColl):
    
    airportdata=Pandas_airport(pd.DataFrame(list(airportColl.find({}))))
    
    airportdata.dropTypeClose()
    airportdata.airportTypeColumns()
    airportdata.marker('small_airport')
    airportdata.marker('heliport')
    airportdata.marker('seaplane_base')
    airportdata.marker('balloonport')
    airportdata.marker('medium_airport')
    airportdata.marker('large_airport')
    
    
    airportdata=airportdata.sizeNGB_sec()
    airportdata.drop(columns=["index"],inplace=True)
    
    freqdata=Pandas_freq(pd.DataFrame(list(freqcoll.find({}))))
    removingDuplicates=freqdata.indexSet()
    removingDuplicates['duplicates']=removingDuplicates.index+removingDuplicates['frequency_mhz']
    removingDuplicates.drop_duplicates(subset=['duplicates'],inplace=True)
    #here the index and  the frequency column are selected for mereging
    freqselect=removingDuplicates['frequency_mhz']
 
    
    toMerged=MergingAnF(airportdata,freqselect)
    mergedData=Copying_freq(toMerged.mergeAnF())
    mergedData.copyfreq('small_airport')
    mergedData.copyfreq('medium_airport')
    mergedData.copyfreq('large_airport')


    surrogateKey=Mongo_prep()
    mergedDBtoDB=surrogateKey.surrogateKey(mergedData.returnDataFrame())
               
 
    try:
        preppedColl.delete_many({})
        toDB(coll,mergedDBtoDB.to_dict('records'))
        showinfo("Executed","Data prepared")
    except:
        showinfo("Error","Please contact administartor")

def cleaningNmergingRunway(runwayColl=runwayColl,db=aviationDB):
    freq_airport=pd.DataFrame(list(preppedColl.find({})))
    runsize=pd.DataFrame(list(runwayColl.find({})))
    
    runsize["unique?"]=(runsize["airport_ident"])+(runsize["le_ident"])
    runsize.drop_duplicates(subset=["unique?"],inplace=True)
    runsize=runsize.filter(['airport_ident','length_ft','width_ft','unique?'])
   
    toMerged=MergingAnF(freq_airport,runsize)
    runway=toMerged.mergeAnF()
   
    runway.drop_duplicates(subset=["unique?"],inplace=True)
    runway.drop(runway.index[runway['type']=='heliport'],inplace=True)

    runway['width_ft'].replace('', np.nan, inplace=True)
    runway['length_ft'].replace('', np.nan, inplace=True)
    runway.dropna(subset=["length_ft"],inplace=True)
    runway.dropna(subset=["width_ft"],inplace=True)
    runway=runway.astype({'length_ft':"float"})
    runway=runway.astype({'width_ft':"float"})
    runway['area']=runway['length_ft']*runway['width_ft']
    runway_totalArea=runway[['ident','unique?','area']]
    runway.drop(columns=['area'],inplace=True)
    test2=runway_totalArea.groupby('ident').sum()
    runway=pd.merge(runway,test2,left_on='ident',right_on='ident',how='left')
    
    runway.rename(columns={'_id':'Foreign_ID_AirportFreq'},inplace=True)
    runway.rename(columns={'unique?':'_id'},inplace=True)
    

    try:
        preppedCollincRunway.delete_many({})
        toDB(preppedCollincRunway,runway.to_dict('records'))
        showinfo("Executed","Data prepared")
    except:
        pass
        showinfo("Error","Please contact administartor")
    
    
    
##This block is used to backup the prepped in .json format

def backup_coll():
    prepped=preppedColl.find({})
    save_prepped=dumps(prepped)
    save=Js_func(file=save_prepped,filename="GBAirpotFreq")
    save.js_backup()

def restore_coll(path=None):
    try:
        restore_file=Js_func(file=path,filename=None)
        file=restore_file.js_restore()
        toDB(preppedColl,restore_file.js_restore())
        showinfo("Restored","Collection restored")
    except FileNotFoundError:
        showinfo("Error","No file selected or file not found ")

##Q3ai of the brief 
def largeMMM(df):
    centralT=df[['ident','large_airport']]
    centralT=centralT['large_airport'].replace('', np.nan)
    centralT=centralT.dropna()
    typechange=centralT.astype({'large_airport':"float"})
    centralTendacy=[]
    centralTendacy.append(typechange.mean())
    centralTendacy.append(typechange.mode())
    centralTendacy.append(typechange.median())
    return centralTendacy

##Q3aii of the brief 
def freqmoreHundred(df):
    df.dropna(subset = ['frequency_mhz'], inplace=True)
    dffloat=df.astype({'frequency_mhz':"float"})
    dffloat.drop(dffloat.index[dffloat['frequency_mhz'] <=100], inplace=True)
    dffloat=dffloat.filter(['frequency_mhz'])
    centralTendacy= pd.DataFrame()
    centralTendacy['Mean'] = np.around(dffloat.mean(), decimals=2)
    centralTendacy['Median'] = np.around(dffloat.median(), decimals=2)
    centralTendacy['Mode'] =np.around( dffloat.iloc[0], decimals=2)
    return centralTendacy

##Q4
def smallairgraph(df):
    df['small_airport'].replace('', np.nan, inplace=True)
    df.dropna(subset = ['small_airport'], inplace=True)
    grouping=df.astype({'small_airport':"float"})
    grouping['quartiles']=pd.cut(grouping['small_airport'],50)
    grouped=grouping.groupby('quartiles').count()
    grouped.reset_index(inplace=True)
    grouped=grouped[['quartiles','small_airport']]
    grouped=grouped.astype({'quartiles':"string"})
    return grouped

    

### Data Analysis

In [8]:
def freq_smallairport():
    
    window.geometry("1000x800")
    grapFrame=Frame(window)
    
    df=smallairgraph(pd.DataFrame(list(preppedColl.find({}))))
    graph=plt.figure()
    small_airportdist=sns.barplot(x=df['quartiles'],y=df['small_airport'],color='black')
    small_airportdist.set(xlabel='Ranges',ylabel='Frequency',title='Small Airport Com Freq')
    txt="The above graph shows the communications frequencies used by samll airports.\n The frequencies are grouped into ranges for ease of display"
    plt.figtext(0.5, 0.2, txt, wrap=True, horizontalalignment='center', fontsize=12)
    plt.xticks(rotation=70)
    plt.tight_layout()
        
    canvas=FigureCanvasTkAgg(graph,master=grapFrame)
    canvas.draw()
    canvas.get_tk_widget().place(x=100,y=20,width=800,height=700)
    
    Button(grapFrame,text="<< Back",bg='yellow',command=lambda:[stats(),grapFrame.destroy()]).place(x=50,y=700,width=120,height=50)
    
    grapFrame.place(x=10,y=20,width=900,height=800)
    
    
def sizeVsFreq():
    window.geometry("900x900")
    grapFrame=Frame(window)
    
    df=pd.DataFrame(list(preppedCollincRunway.find({})))
    toplot=df[['area','frequency_mhz']]
    toplot=toplot.fillna(0)
    toplot=toplot.astype({'frequency_mhz':"float"})
    corrval=toplot.corr()
    
    graph=plt.figure(figsize = (15,15))
    freqregplot=sns.regplot(x='area',y='frequency_mhz',data=toplot,)
    freqregplot.set(xlabel='Airport size',ylabel='Frequency in mhz size',title='Small Airport Com Freq')
    txt="The above graph shows the communications frequencies used by samll airports.\n The frequencies are grouped into ranges for easy of display\n Correlation Coefficient: {:.3f}".format(corrval["frequency_mhz"][0])
    plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=12)
    
    canvas=FigureCanvasTkAgg(graph,master=grapFrame)
    canvas.draw()
    canvas.get_tk_widget().place(x=100,y=20,width=700,height=700)
    
    Button(grapFrame,text="<< Back",bg='yellow',command=lambda:[stats(),grapFrame.destroy()]).place(x=50,y=745,width=120,height=50)
    
    grapFrame.place(x=10,y=20,width=1000,height=1000)
    
def allairports_MMM():
    window.geometry("400x250")
    grapFrame=Frame(window)
    
    meanText=Text(grapFrame, height = 2, width = 10,font=15)
    medianText=Text(grapFrame, height = 2, width = 10,font=15)
    modeText=Text(grapFrame, height = 2, width = 10,font=15)
    
    description=Label(grapFrame,text="The following data shows the central \ntendency of all available Frequencies")
    description.config(font=15)
    meanlabel=Label(grapFrame,text="Mean")
    meanlabel.config(font=15)
    medianlabel=Label(grapFrame,text="Median")
    medianlabel.config(font=15)
    modelabel=Label(grapFrame,text="Mode")
    modelabel.config(font=15)

    
    mmm=freqmoreHundred(pd.DataFrame(list(preppedColl.find({}))))
    
    mean=mmm['Mean'][0]
    median=mmm['Median'][0]
    mode=mmm['Mode'][0]
    
    
    description.pack()
    meanlabel.pack() 
    meanText.pack()
    medianlabel.pack() 
    medianText.pack()
    modelabel.pack() 
    modeText.pack()
    
    meanText.insert('end',mean)
    meanText.config(state='disabled')
    medianText.insert('end',median)
    medianText.config(state='disabled')
    modeText.insert('end',mode)
    modeText.config(state='disabled')
    
    description.place(x=5,y=100)
    meanlabel.place(x=20,y=20)
    meanText.place(x=10,y=50)
    medianlabel.place(x=140,y=20)
    medianText.place(x=120,y=50)
    modelabel.place(x=260,y=20)
    modeText.place(x=240,y=50)
    
    Button(grapFrame,text="<< Back",bg='yellow',command=lambda:[stats(),grapFrame.destroy()]).place(x=100,y=170,width=120,height=50)
    
    grapFrame.place(x=10,y=20,width=1000,height=1000)

def largeairports_MMM():
    window.geometry("400x250")
    grapFrame=Frame(window)
    
    meanText=Text(grapFrame, height = 2, width = 10,font=15)
    medianText=Text(grapFrame, height = 2, width = 10,font=15)
    modeText=Text(grapFrame, height = 2, width = 10,font=15)
    
    description=Label(grapFrame,text="The following data shows the central \ntendency of all Large airports")
    description.config(font=15)
    meanlabel=Label(grapFrame,text="Mean")
    meanlabel.config(font=15)
    medianlabel=Label(grapFrame,text="Median")
    medianlabel.config(font=15)
    modelabel=Label(grapFrame,text="Mode")
    modelabel.config(font=15)
   
    mmm=largeMMM(pd.DataFrame(list(preppedColl.find({}))))
    
    mean=np.round(mmm[0],decimals=2)
    median=np.round(mmm[2],decimals=2)
    mode=np.round(mmm[1][0],decimals=2)
    
    
    description.pack()
    meanlabel.pack() 
    meanText.pack()
    medianlabel.pack() 
    medianText.pack()
    modelabel.pack() 
    modeText.pack()
    
    meanText.insert('end',mean)
    meanText.config(state='disabled')
    medianText.insert('end',median)
    medianText.config(state='disabled')
    modeText.insert('end',mode)
    modeText.config(state='disabled')
    
    description.place(x=5,y=100)
    meanlabel.place(x=20,y=20)
    meanText.place(x=10,y=50)
    medianlabel.place(x=140,y=20)
    medianText.place(x=120,y=50)
    modelabel.place(x=260,y=20)
    modeText.place(x=240,y=50)
    
    Button(grapFrame,text="<< Back",bg='yellow',command=lambda:[stats(),grapFrame.destroy()]).place(x=100,y=170,width=120,height=50)
    
    grapFrame.place(x=10,y=20,width=1000,height=1000)
    

### tkinter process for the visualisation

In [9]:
def stats():
    
    window.geometry("300x300")
    grapMFrame=Frame(window)
        
    
    Button(grapMFrame,text="Small Airport Com frequencies",command=lambda : [freq_smallairport(),grapMFrame.destroy()]).grid(row=1,column=1,sticky='NESW', pady=7,padx=2)
    Button(grapMFrame,text="Size of Airport Vs Com frequencies",command=lambda : [sizeVsFreq(),grapMFrame.destroy()]).grid(row=2,column=1,sticky='NESW', pady=7,padx=2)
    Button(grapMFrame,text="All Freqs Mean,Median & Mode ",command=lambda : [allairports_MMM(),grapMFrame.destroy()]).grid(row=3,column=1,sticky='NESW', pady=7,padx=2)
    Button(grapMFrame,text="Large Airport Freqs Mean,Median & Mode ",command=lambda : [largeairports_MMM(),grapMFrame.destroy()]).grid(row=4,column=1,sticky='NESW', pady=7,padx=2)
    
    mmButton(grapMFrame,grapMFrame).grid(row=5,column=1,sticky='NESW', pady=7,padx=2)
    
    grapMFrame.place(x=10,y=20,width=1000,height=1000)

In [10]:
def browseFiles_csv():
        filetypes = (
        ('csv files', '*.csv'),
        ('All files', '*.*'))
        filename = fd.askopenfilename(title='Open a file', initialdir=os.getcwd() ,filetypes=filetypes)
        if filename == None:
            return "None"
        else:
            return filename
        
def browseFiles_json():
        filetypes = (
        ('json files', '*.json'),
        ('All files', '*.*'))
        filename = fd.askopenfilename(title='Open a file', initialdir=os.getcwd(),filetypes=filetypes)
        if filename == None:
            return "None"
        else:
            return filename


In [11]:
def import_data():
    
    window.title("Mongo Import")
    window.geometry("400x250")
    importDataWindow=Frame(window)
    
    aiportimport=StringVar()
    aiportimport.set("Please click to select location of airport data: ")
    freqimport=StringVar()
    freqimport.set("Please click to select location of frequency data: ")
    runimport=StringVar()
    runimport.set("Please click to select location of runway data: ")
    
    Label(importDataWindow,text=aiportimport.get()).grid(row=1,column=1,sticky='W', pady=7,padx=2)
    Label(importDataWindow,text=freqimport.get()).grid(row=2,column=1,sticky='W', pady=7,padx=2)
    Label(importDataWindow,text=runimport.get()).grid(row=3,column=1,sticky='W', pady=7)
    
    #https://www.geeksforgeeks.org/file-explorer-in-python-using-tkinter/
    #https://www.pythontutorial.net/tkinter/tkinter-open-file-dialog/
    airport_import=Button(importDataWindow,text="Import",command=lambda:[airportData(browseFiles_csv())]).grid(row=1,column=2,sticky='W', pady=7,padx=2)
    freq_import=Button(importDataWindow,text="Import",command=lambda:[freqData(browseFiles_csv())]).grid(row=2,column=2,sticky='W', pady=7,padx=2)
    runway_import=Button(importDataWindow,text="Import",command=lambda:[runwayData(browseFiles_csv())]).grid(row=3,column=2,sticky='W', pady=7,padx=2)                                                                    
    
    
    mmButton(importDataWindow,importDataWindow).grid(row=5,column=1,sticky='NESW', pady=7,padx=2)
    
    
    importDataWindow.place(x=10,y=20,width=400,height=560)
                                                       

    
    

In [12]:
def restoreTK():
    window.title("Mongo Restore")
    window.geometry("400x200")
    restoreWindow=Frame(window)
    
    messagebox.showwarning("Waring!!","All data is will be deleted Prepped collection. To carry on click execute")
    
    restoremessage=StringVar()
    restoremessage.set("Please select the location of the Prepped data you would like to restore: ")

    
    restoreL=Label(restoreWindow,text=restoremessage.get()).grid(row=1,column=1,sticky='NESW', pady=7,padx=2)
    Button(restoreWindow,text="Restore",command=lambda:[restore_coll(browseFiles_json())]).grid(row=2,column=1,sticky='NESW')
    
    Button(restoreWindow,text="<< Back",bg='yellow',command=lambda:[prepped_data(),restoreWindow.destroy()]).place(x=150,y=70,width=120,height=50)
    
    restoreWindow.place(x=10,y=20,width=400,height=560)

In [13]:
def prepped_data():
    window.title("Mongo Data Prep")
    window.geometry("300x300")
    backupPrepWindow=Frame(window)
    
    Button(backupPrepWindow,text="Prep Freq and Airprot Data",command=lambda:[cleaningNmerging()]).grid(row=1,column=1,sticky='NESW', pady=7,padx=2)
    Button(backupPrepWindow,text="Prep runway",command=lambda:[cleaningNmergingRunway()]).grid(row=2,column=1,sticky='NESW', pady=7,padx=2)
    Button(backupPrepWindow,text="Backup prepped Data",command=lambda:[backup_coll()]).grid(row=3,column=1,sticky='NESW', pady=7,padx=2)
    Button(backupPrepWindow,text="Restore prepped Data",command=lambda:[restoreTK(),backupPrepWindow.destroy()]).grid(row=4,column=1,sticky='NESW', pady=7,padx=2)
    
    
    ##button to go back to the main menu
    mmButton(backupPrepWindow,backupPrepWindow).grid(row=5,column=1,sticky='NESW', pady=7,padx=2)
    
    
    backupPrepWindow.place(x=10,y=20,width=400,height=560)

In [14]:
def mmButton(x=None,frame=None):
    mainMenuButton=Button(x,text="<< Main Menu",bg='yellow',command=lambda:[main_Menu(),frame.destroy()])
    return mainMenuButton


def main_Menu():
    window.geometry("300x300")
    mainMenu=Frame(window)
    
    Button(mainMenu,text="Data import",command=lambda:[import_data(),mainMenu.destroy()]).grid(row=1,column=1,sticky='NESW', pady=7,padx=2)
    Button(mainMenu,text="Data prep",command=lambda:[prepped_data(),mainMenu.destroy()]).grid(row=2,column=1,sticky='NESW', pady=7,padx=2)
    Button(mainMenu,text="Descriptive Stats",command=lambda:[stats(),mainMenu.destroy()]).grid(row=3,column=1,sticky='NESW', pady=7,padx=2)
    Button(mainMenu,text="Quit",command=lambda:[window.destroy()],bg='red').grid(row=5,column=1,sticky='NESW', pady=7,padx=2)
    
    
    mainMenu.place(x=10,y=20,width=150,height=560)

In [15]:
window =Tk()
window.title("Main Menu")
window.geometry("")

main_Menu()


window.mainloop()